In [1]:
import os
import csv
import numpy as np
from numpy import genfromtxt
import pandas as pd
import random
import time
import os.path

In [28]:
def read_subs_dataframe():
    subs=pd.read_csv('Data/subs.csv', sep=',',header=None,encoding="utf-8");
    return subs

In [22]:
def read_subs():
    data_path='Data/';
    data = os.listdir(data_path);
    for files in data:
        with open(os.path.join(data_path,files),newline='',encoding="utf-8") as f:

            rows=csv.reader(f, delimiter=',');
            if(files=='subs.csv'):
                sub=[];
                for row in rows:
                    sub.append([row[0],list(map(int, row[1:]))]);
    return sub;

In [23]:
def read_key():
    key=pd.read_csv('Data/key_2.csv', sep=',',header=None,encoding="utf-8")
    return key;

In [33]:
key_file = 'Data/keys2.pkl'
if os.path.exists(key_file):
    print('Reading processed keys...');
    pkeys =pd.read_pickle(key_file);
else:
    print('Reading keys...')
    raw_keys = pd.read_csv('Data/key_2.csv')
    print('Processing keys...')
    pagedate = raw_keys.Page.str.rsplit('_', expand=True, n=1).rename(columns={0:'page',1:'date_str'})
    keys = raw_keys.drop('Page', axis=1).assign(page=pagedate.page, date=pd.to_datetime(pagedate.date_str))
    del raw_keys, pagedate
    print('Pivoting keys...')
    pkeys = keys.pivot(index='page', columns='date', values='Id')
    print('Storing keys...')
    pkeys.to_pickle(key_file)

Reading processed keys...


In [25]:
def prediction_date():
    start = '2017-09-13'
    end = '2017-11-13'
    d = pd.date_range(start, end)
    d.strftime('%Y-%m-%d');
    date=[];
    for a in d:
        date.append('_'+str(a)[:-9]);
    return date

In [26]:
def extract_view(subs):
    daily_view=np.zeros([len(subs),len(date)]);
    for i in range(np.shape(daily_view)[0]):
        for j in range(np.shape(daily_view)[1]):
            daily_view[i,j-1]=subs[i][1][j];
    return daily_view;

In [29]:
ID_and_view=read_subs_dataframe();

In [30]:
key=read_key();

In [31]:
date=prediction_date();

In [31]:
#daily_view=extract_view(ID_and_view);

In [32]:
ID=ID_and_view.values[:,0];

In [83]:
num_prediction=62;
Id=[];
Visit=[];
start_time = time.time();
v=ID_and_view.values;
for i in range(len(v)):#93,103#len(v)
    #
    index=np.where(pkeys.index==v[i,0])[0][0];   
    for j in range(num_prediction):
        Id.append(pkeys.values[index][j]);
        Visit.append(v[i][j+1]);
    #print(i);
    if(i%1000==0):
        print(i);
        print("--- %s seconds ---" % (time.time() - start_time));

0
--- 0.35928916931152344 seconds ---
1000
--- 21.78899574279785 seconds ---
2000
--- 45.042994260787964 seconds ---
3000
--- 63.95900344848633 seconds ---
4000
--- 84.52691769599915 seconds ---
5000
--- 102.78518056869507 seconds ---
6000
--- 120.3698194026947 seconds ---
7000
--- 138.89639139175415 seconds ---
8000
--- 157.95236897468567 seconds ---
9000
--- 177.2478096485138 seconds ---
10000
--- 196.07381749153137 seconds ---
11000
--- 215.1975383758545 seconds ---
12000
--- 234.6401674747467 seconds ---
13000
--- 254.55269241333008 seconds ---
14000
--- 273.2266893386841 seconds ---
15000
--- 292.13075375556946 seconds ---
16000
--- 313.34115505218506 seconds ---
17000
--- 331.61555099487305 seconds ---
18000
--- 351.6671211719513 seconds ---
19000
--- 372.933869600296 seconds ---
20000
--- 393.31057810783386 seconds ---
21000
--- 413.2352056503296 seconds ---
22000
--- 435.3064138889313 seconds ---
23000
--- 458.0060935020447 seconds ---
24000
--- 480.39341950416565 seconds ---
2

In [84]:
with open('Data/submission_2.csv', 'w', newline='') as csvfile:
    spamwriter = csv.writer(csvfile, delimiter=',');
    spamwriter.writerow(['Id','Visits']);
    for i in range(len(Visit)):
        spamwriter.writerow([Id[i],Visit[i]]);